In [3]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
df = pd.read_csv('Tweets.csv')

In [4]:
def normalizer(tweet):
      only_letters = re.sub("[^a-zA-Z]", " ", tweet)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
nltk.download('wordnet')
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\musfi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [8]:
train_x,val_x,train_y,val_y = train_test_split(x_vectorized,y)

In [11]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(train_x, train_y)


ValueError: sparse multilabel-indicator for y is not supported.

In [9]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(train_x, train_y)
model = gs_clf.best_estimator_

In [10]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [15]:
print(y_pred)
print(_statistics)

['negative' 'negative' 'positive' ... 'negative' 'positive' 'negative']
{'f1_score': 0.7811475409836065, 'confusion_matrix': array([[2013,  181,   67],
       [ 275,  457,   74],
       [ 130,   74,  389]], dtype=int64), 'precision': 0.7811475409836065, 'recall': 0.7811475409836065}


In [31]:
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])
model.predict(test_feature)

array(['negative'], dtype=object)

In [18]:
test_feature = vectorizer.transform(['Movie is good'])
model.predict(test_feature)

array(['positive'], dtype=object)

In [19]:
test_feature = vectorizer.transform(['I\'m okay'])
model.predict(test_feature)

array(['neutral'], dtype=object)

In [20]:
import pickle

In [21]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models'+".p", "wb"))

In [28]:
with open('models.p', 'rb') as pickled:
    data = pickle.load(pickled)

model = data['model']
vectorizer = data['vectorizer']

In [41]:
text = "I am a bad boy"

In [42]:
vector = vectorizer.transform([text])
prediction = model.predict(vector)[0]

In [43]:
prediction

'negative'